In [1]:
import aop
import aop.api
import json
import time

import pymysql
import pandas as pd
import numpy as np
import datetime
from urllib.parse import unquote

#google sheet 專用   #pip install google-api-python-client
import string
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [12]:
# 時間日期設定
today=(datetime.datetime.now() - datetime.timedelta(days=0)).strftime("%Y-%m-%d") 
# 存檔案路徑
downloadpath='//navdi/DEM_UserData/btorin/Desktop/download_csv/'
# 设置网关域名
aop.set_default_server('gateway.open.umeng.com')
# 设置apiKey和apiSecurity
aop.set_default_appinfo(6560937, "icwdcKfDDGY")
# appkey_channel
appkey_h5_new='6320305405844627b54716ca'

In [13]:
#統計啟動次數 活躍 新增
req = aop.api.UmengUminiGetOverviewRequest()
resp_count = req.get_response(None, dataSourceId=appkey_h5_new, fromDate=today, toDate=today, timeUnit="day", indicators="visitTimes", pageIndex=1, pageSize=1)
resp_active = req.get_response(None, dataSourceId=appkey_h5_new, fromDate=today, toDate=today, timeUnit="day", indicators="activeUser", pageIndex=1, pageSize=1)
resp_new = req.get_response(None, dataSourceId=appkey_h5_new, fromDate=today, toDate=today, timeUnit="day", indicators="newUser", pageIndex=1, pageSize=1)
#轉換
resp_count=resp_count['data']['data'][0]  ; resp_active=resp_active['data']['data'][0]  ; resp_new=resp_new['data']['data'][0]
resp_count["value"] = resp_count.pop("visitTimes")  ;resp_active["value"] = resp_active.pop("activeUser")  ; resp_new["value"] = resp_new.pop("newUser")
resp_total=[resp_count,resp_active,resp_new]
resp_total=pd.DataFrame(resp_total)
resp_total['item']=['启动次数','活跃会员','新增用户']
resp_total['device']='H5'
dd_data_h5=resp_total[['dateTime','device','item','value']]
dd_data_h5.columns=['date','device','item','value']

In [40]:
#事件點擊 統計
req1 = aop.api.UmengUminiGetEventOverviewRequest()
resp1 = req1.get_response(None, timeUnit="day", fromDate=today, toDate=today, eventName="search_total", dataSourceId=appkey_h5_new)
resp1_data=pd.DataFrame(resp1['data']['data'][0] , index=[0])
resp1_data['eventname']='搜尋總計'

req2 = aop.api.UmengUminiGetEventOverviewRequest()
resp2 = req2.get_response(None, timeUnit="day", fromDate=today, toDate=today, eventName="search_success", dataSourceId=appkey_h5_new)
resp2_data=pd.DataFrame(resp2['data']['data'][0] , index=[0])
resp2_data['eventname']='成功搜尋次數'

#合併
resp_data_total=resp1_data.append(resp2_data)

In [46]:
resp_data_total

,dateTime,count,device,eventname
0,2022-09-15,9,7,搜尋總計
0,2022-09-15,5,4,成功搜尋次數


#  寫入雲端表格

In [448]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly',
      'https://www.googleapis.com/auth/drive',
      'https://www.googleapis.com/auth/spreadsheets']
creds = None
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)
service = build('sheets', 'v4', credentials=creds)
# Call the Sheets API
sheet = service.spreadsheets()

#寫入 data 進入 google sheet #美東時間
def insert_data(value_range_body,sheetname,colname1,colname2,col1,col2) :
    SAMPLE_RANGE_NAME = str(sheetname)+'!'+colname1+str(col1)+':'+colname2+str(col2)
    request= sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,valueInputOption='USER_ENTERED',range=SAMPLE_RANGE_NAME , body=value_range_body).execute()
def google_sheet_insert_data(sheet_name,data,SAMPLE_SPREADSHEET_ID) : 
    #查詢目前使用到維度
    SAMPLE_RANGE_NAME = sheet_name+'!A1:A30000'
    result_index = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME).execute()
    insert_row=len(result_index.get('values'))+1
    #遊戲驗證
    value_range_body = {"majorDimension":"ROWS","values": data.values.tolist() }
    insert_data(value_range_body,sheet_name,'A','D',insert_row,insert_row+50)
    
def google_sheet_insert_data_event(sheet_name,data,SAMPLE_SPREADSHEET_ID) : 
    #查詢目前使用到維度
    SAMPLE_RANGE_NAME = sheet_name+'!A1:A70000'
    result_index = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME).execute()
    insert_row=len(result_index.get('values'))+1
    #遊戲驗證
    value_range_body = {"majorDimension":"ROWS","values": data.values.tolist() }
    insert_data(value_range_body,sheet_name,'A','E',insert_row,insert_row+500)

In [449]:
#流量數據
SAMPLE_SPREADSHEET_ID = '14OR2-WaGLyZfFpMWbxs_g7sureaZYxCb0XKKMmpMZpU'
google_sheet_insert_data('會員數據',dd_data,SAMPLE_SPREADSHEET_ID)
#外層事件數據
google_sheet_insert_data_event('外层事件',dd_event1,SAMPLE_SPREADSHEET_ID)
